In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Concatenate, Lambda, Add, Dot, Activation, Layer

In [ ]:
from utils.conf_process import myconfigparser
from utils.feature_process import tuple_feature_process
from utils.active_method import activate
from utils.optimizer_method import get_optimizer
from utils.distribut_conf import set_dist_env

In [ ]:
def build_model_columns(conf_path):

    deep_columns = list()
    default_dict = {}
    embedding_nums = 0

    cfg = myconfigparser()
    cfg.read(conf_path)
    sections = cfg.sections()

    for part_key in sections:

        items_lst = cfg.items(part_key)

        if part_key == "deep":
            for items in items_lst:
                feature_name = items[0].strip()
                feat_method,default_val,embedding_num = tuple_feature_process(items,int(FLAGS.embedding_size))
                embedding_nums += embedding_num
                if feature_name not in default_dict:
                    default_dict[feature_name] = default_val
                deep_columns.append(feat_method)

    columns = cfg.items("use")[0][1].strip().split(",")
    default_dict["label"] = 0
    cols_default = list(map(lambda x:["-1"] if x not in default_dict else [default_dict[x]], columns))
    return deep_columns, columns, cols_default,embedding_nums


In [ ]:
fixlen_feature_columns = [DenseFeat(feat, 1,) for feat in dense_features] \
                       + [SparseFeat(feat, data[feat].nunique()) for feat in sparse_features
                             if data[feat].nunique() < 10000] \

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

fixlen_feature_names = get_fixlen_feature_names(
    linear_feature_columns + dnn_feature_columns)

In [ ]:
class FM(tf.keras.layers.Layer):
    """Factorization Machine models pairwise (order-2) feature interactions
     without linear term and bias.
      Input shape
        - 3D tensor with shape: ``(batch_size,field_size,embedding_size)``.
      Output shape
        - 2D tensor with shape: ``(batch_size, 1)``.
    """

    def __init__(self):
        super(FM, self).__init__()

    def call(self, input):
        square_of_sum = tf.math.pow(tf.math.reduce_sum(input, 1, keepdims=True), 2)
        sum_of_square = tf.math.reduce_sum(input * input, 1, keepdims=True)
        cross_term = square_of_sum - sum_of_square
        cross_term = 0.5 * tf.math.reduce_sum(cross_term, axis=2, keepdims=False)

        return tf.squeeze(cross_term, -1)
